In [ ]:
import pandas as pd
from numpy import random

import eli5
from eli5.sklearn import PermutationImportance

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


import shap  # package used to calculate Shap values


import gc
import random as python_random
import time


/home/umesh/.local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/umesh/.local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
data = pd.read_csv('./final_project_data/sales_train.csv')
category = pd.read_csv('./final_project_data/item_categories.csv')
items = pd.read_csv('./final_project_data/items.csv')

# data = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv')
# category = pd.read_csv('../input/competitive-data-science-predict-future-sales/item_categories.csv')
# items = pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv')

In [ ]:
category['split'] = category['item_category_name'].str.split('-')
category['type'] = category['split'].map(lambda x: x[0].strip())
category['type_code'] = LabelEncoder().fit_transform(category['type'])

category['subtype'] = category['split'].map(lambda x: x[1].strip() if len(x) > 1 else x[0].strip())
category['subtype_code'] = LabelEncoder().fit_transform(category['subtype'])
category = category[['item_category_id','type_code', 'subtype_code']]

item_category_mapping = items[['item_id','item_category_id']].drop_duplicates()


In [ ]:
data = pd.merge(data, item_category_mapping, how='left', on='item_id')

data = pd.merge(data, category, on=['item_category_id'], how='left')



In [ ]:
list =[]
for i in range(data.shape[0] // 100):
    list.append(i*100 )

data = data.iloc[list]
data.shape

In [ ]:
base_features = [c for c in data.columns if ((c != "item_cnt_day") & (c != "date"))]
X = data[base_features]
y = data['item_cnt_day'].rename('target')

X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=1)


model = RandomForestClassifier(n_estimators=10, random_state=1).fit(X_train, y_train)

In [ ]:
perm = PermutationImportance(model, random_state=1).fit(X_val, y_val)
eli5.show_weights(perm, feature_names = X_val.columns.tolist())

In [ ]:
sample_data_for_prediction = X_train.iloc[222].astype(float)  # to test function
data_for_prediction_array = sample_data_for_prediction.values.reshape(1, -1)


def imp_feature(model, sale_data):
    # Create object that can calculate shap values
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(sale_data)
    shap.initjs()
    return shap.force_plot(explainer.expected_value[1], shap_values[1], sale_data)


In [ ]:
imp_feature(model,data_for_prediction_array)

In [ ]:
data.columns
